In [3]:
myarg= '--data_dir dataset/tacred --vocab_dir dataset/vocab --id 00 --info "Position-aware attention model"'


In [12]:
"""
Train a model on TACRED.
"""

import os
from datetime import datetime
import time
import numpy as np
import random
import argparse
from shutil import copyfile
import torch
import torch.nn as nn
import torch.optim as optim

from data.loader import DataLoader
from model.rnn import RelationModel
from utils import scorer, constant, helper
from utils.vocab import Vocab

parser = argparse.ArgumentParser()
#"""
parser.add_argument('--data_dir', type=str, default='dataset/tacred')
parser.add_argument('--vocab_dir', type=str, default='dataset/vocab')
parser.add_argument('--emb_dim', type=int, default=300, help='Word embedding dimension.')
parser.add_argument('--ner_dim', type=int, default=30, help='NER embedding dimension.')
parser.add_argument('--pos_dim', type=int, default=30, help='POS embedding dimension.')
parser.add_argument('--hidden_dim', type=int, default=200, help='RNN hidden state size.')
parser.add_argument('--num_layers', type=int, default=2, help='Num of RNN layers.')
parser.add_argument('--dropout', type=float, default=0.5, help='Input and RNN dropout rate.')
parser.add_argument('--word_dropout', type=float, default=0.04, help='The rate at which randomly set a word to UNK.')
parser.add_argument('--topn', type=int, default=1e10, help='Only finetune top N embeddings.')
parser.add_argument('--lower', dest='lower', action='store_true', help='Lowercase all words.')
parser.add_argument('--no-lower', dest='lower', action='store_false')
parser.set_defaults(lower=False)

parser.add_argument('--attn', dest='attn', action='store_true', help='Use attention layer.')
parser.add_argument('--no-attn', dest='attn', action='store_false')
parser.set_defaults(attn=True)
parser.add_argument('--attn_dim', type=int, default=200, help='Attention size.')
parser.add_argument('--pe_dim', type=int, default=30, help='Position encoding dimension.')

parser.add_argument('--lr', type=float, default=1.0, help='Applies to SGD and Adagrad.')
parser.add_argument('--lr_decay', type=float, default=0.9)
parser.add_argument('--optim', type=str, default='sgd', help='sgd, adagrad, adam or adamax.')
parser.add_argument('--num_epoch', type=int, default=30)
parser.add_argument('--batch_size', type=int, default=50)
parser.add_argument('--max_grad_norm', type=float, default=5.0, help='Gradient clipping.')
parser.add_argument('--log_step', type=int, default=20, help='Print log every k steps.')
parser.add_argument('--log', type=str, default='logs.txt', help='Write training log to file.')
parser.add_argument('--save_epoch', type=int, default=5, help='Save model checkpoints every k epochs.')
parser.add_argument('--save_dir', type=str, default='./saved_models', help='Root dir for saving models.')
parser.add_argument('--id', type=str, default='00', help='Model ID under which to save models.')
#parser.add_argument('--info', type=str, default='', help='Optional info for the experiment.')
parser.add_argument('--info', type=str, default='', help='Position-aware attention model')

parser.add_argument('--seed', type=int, default=1234)
parser.add_argument('--cuda', type=bool, default=torch.cuda.is_available())
parser.add_argument('--cpu', action='store_true', help='Ignore CUDA.')
#args = parser.parse_args()
#"""
args = parser.parse_args([str(myarg)])

usage: ipykernel_launcher.py [-h] [--data_dir DATA_DIR]
                             [--vocab_dir VOCAB_DIR] [--emb_dim EMB_DIM]
                             [--ner_dim NER_DIM] [--pos_dim POS_DIM]
                             [--hidden_dim HIDDEN_DIM]
                             [--num_layers NUM_LAYERS] [--dropout DROPOUT]
                             [--word_dropout WORD_DROPOUT] [--topn TOPN]
                             [--lower] [--no-lower] [--attn] [--no-attn]
                             [--attn_dim ATTN_DIM] [--pe_dim PE_DIM] [--lr LR]
                             [--lr_decay LR_DECAY] [--optim OPTIM]
                             [--num_epoch NUM_EPOCH] [--batch_size BATCH_SIZE]
                             [--max_grad_norm MAX_GRAD_NORM]
                             [--log_step LOG_STEP] [--log LOG]
                             [--save_epoch SAVE_EPOCH] [--save_dir SAVE_DIR]
                             [--id ID] [--info INFO] [--seed SEED]
                             [--c

SystemExit: 2

In [11]:
torch.manual_seed(args.seed)
np.random.seed(args.seed)
random.seed(1234)
if args.cpu:
    args.cuda = False
elif args.cuda:
    torch.cuda.manual_seed(args.seed)

# make opt
opt = vars(args)
opt['num_class'] = len(constant.LABEL_TO_ID)

# load vocab
vocab_file = opt['vocab_dir'] + '/vocab.pkl'
vocab = Vocab(vocab_file, load=True)
opt['vocab_size'] = vocab.size
emb_file = opt['vocab_dir'] + '/embedding.npy'
emb_matrix = np.load(emb_file)
assert emb_matrix.shape[0] == vocab.size
assert emb_matrix.shape[1] == opt['emb_dim']

# load data
print("Loading data from {} with batch size {}...".format(opt['data_dir'], opt['batch_size']))
train_batch = DataLoader(opt['data_dir'] + '/train.json', opt['batch_size'], opt, vocab, evaluation=False)
dev_batch = DataLoader(opt['data_dir'] + '/dev.json', opt['batch_size'], opt, vocab, evaluation=True)

model_id = opt['id'] if len(opt['id']) > 1 else '0' + opt['id']
model_save_dir = opt['save_dir'] + '/' + model_id
opt['model_save_dir'] = model_save_dir
helper.ensure_dir(model_save_dir, verbose=True)

# save config
helper.save_config(opt, model_save_dir + '/config.json', verbose=True)
vocab.save(model_save_dir + '/vocab.pkl')
file_logger = helper.FileLogger(model_save_dir + '/' + opt['log'], header="# epoch\ttrain_loss\tdev_loss\tdev_f1")

# print model info
helper.print_config(opt)

# model
model = RelationModel(opt, emb_matrix=emb_matrix)

id2label = dict([(v,k) for k,v in constant.LABEL_TO_ID.items()])
dev_f1_history = []
current_lr = opt['lr']

global_step = 0
global_start_time = time.time()
format_str = '{}: step {}/{} (epoch {}/{}), loss = {:.6f} ({:.3f} sec/batch), lr: {:.6f}'
max_steps = len(train_batch) * opt['num_epoch']

# start training
for epoch in range(1, opt['num_epoch']+1):
    train_loss = 0
    for i, batch in enumerate(train_batch):
        start_time = time.time()
        global_step += 1
        loss = model.update(batch)
        train_loss += loss
        if global_step % opt['log_step'] == 0:
            duration = time.time() - start_time
            print(format_str.format(datetime.now(), global_step, max_steps, epoch,\
                    opt['num_epoch'], loss, duration, current_lr))

    # eval on dev
    print("Evaluating on dev set...")
    predictions = []
    dev_loss = 0
    for i, batch in enumerate(dev_batch):
        preds, _, loss = model.predict(batch)
        predictions += preds
        dev_loss += loss
    predictions = [id2label[p] for p in predictions]
    dev_p, dev_r, dev_f1 = scorer.score(dev_batch.gold(), predictions)
    
    train_loss = train_loss / train_batch.num_examples * opt['batch_size'] # avg loss per batch
    dev_loss = dev_loss / dev_batch.num_examples * opt['batch_size']
    print("epoch {}: train_loss = {:.6f}, dev_loss = {:.6f}, dev_f1 = {:.4f}".format(epoch,\
            train_loss, dev_loss, dev_f1))
    file_logger.log("{}\t{:.6f}\t{:.6f}\t{:.4f}".format(epoch, train_loss, dev_loss, dev_f1))

    # save
    model_file = model_save_dir + '/checkpoint_epoch_{}.pt'.format(epoch)
    model.save(model_file, epoch)
    if epoch == 1 or dev_f1 > max(dev_f1_history):
        copyfile(model_file, model_save_dir + '/best_model.pt')
        print("new best model saved.")
    if epoch % opt['save_epoch'] != 0:
        os.remove(model_file)
    
    # lr schedule
    if len(dev_f1_history) > 10 and dev_f1 <= dev_f1_history[-1] and \
            opt['optim'] in ['sgd', 'adagrad']:
        current_lr *= opt['lr_decay']
        model.update_lr(current_lr)

    dev_f1_history += [dev_f1]
    print("")

print("Training ended with {} epochs.".format(epoch))



Vocab size 421 loaded from file
Loading data from dataset/tacred with batch size 50...
1 batches created for dataset/tacred/train.json
1 batches created for dataset/tacred/dev.json
Config saved to file ./saved_models/00/config.json
Overwriting old vocab file at ./saved_models/00/vocab.pkl

Running with the following configs:
	data_dir : dataset/tacred
	vocab_dir : dataset/vocab
	emb_dim : 300
	ner_dim : 30
	pos_dim : 30
	hidden_dim : 200
	num_layers : 2
	dropout : 0.5
	word_dropout : 0.04
	topn : 10000000000.0
	lower : False
	attn : True
	attn_dim : 200
	pe_dim : 30
	lr : 1.0
	lr_decay : 0.9
	optim : sgd
	num_epoch : 30
	batch_size : 50
	max_grad_norm : 5.0
	log_step : 20
	log : logs.txt
	save_epoch : 5
	save_dir : ./saved_models
	id : 00
	info : 
	seed : 1234
	cuda : False
	cpu : False
	num_class : 42
	vocab_size : 421
	model_save_dir : ./saved_models/00


Finetune all embeddings.
Evaluating on dev set...
Precision (micro): 100.000%
   Recall (micro): 0.000%
       F1 (micro): 0.000%


In [13]:
"""
Run evaluation with saved models.
"""

import os
import random
import argparse
import pickle
import torch
import torch.nn as nn
import torch.optim as optim

from data.loader import DataLoader
from model.rnn import RelationModel
from utils import torch_utils, scorer, constant, helper
from utils.vocab import Vocab

parser = argparse.ArgumentParser()
parser.add_argument('model_dir', type=str, help='Directory of the model.')
parser.add_argument('--model', type=str, default='best_model.pt', help='Name of the model file.')
parser.add_argument('--data_dir', type=str, default='dataset/tacred')
parser.add_argument('--dataset', type=str, default='test', help="Evaluate on dev or test.")
parser.add_argument('--out', type=str, default='', help="Save model predictions to this dir.")

parser.add_argument('--seed', type=int, default=1234)
parser.add_argument('--cuda', type=bool, default=torch.cuda.is_available())
parser.add_argument('--cpu', action='store_true')
args = parser.parse_args(["saved_models/00"])

torch.manual_seed(args.seed)
random.seed(1234)
if args.cpu:
    args.cuda = False
elif args.cuda:
    torch.cuda.manual_seed(args.seed)

# load opt
model_file = args.model_dir + '/' + args.model
print("Loading model from {}".format(model_file))
opt = torch_utils.load_config(model_file)
model = RelationModel(opt)
model.load(model_file)

# load vocab
vocab_file = args.model_dir + '/vocab.pkl'
vocab = Vocab(vocab_file, load=True)
assert opt['vocab_size'] == vocab.size, "Vocab size must match that in the saved model."

# load data
data_file = opt['data_dir'] + '/{}.json'.format(args.dataset)
print("Loading data from {} with batch size {}...".format(data_file, opt['batch_size']))
batch = DataLoader(data_file, opt['batch_size'], opt, vocab, evaluation=True)

helper.print_config(opt)
id2label = dict([(v,k) for k,v in constant.LABEL_TO_ID.items()])

predictions = []
all_probs = []
for i, b in enumerate(batch):
    preds, probs, _ = model.predict(b)
    predictions += preds
    all_probs += probs
predictions = [id2label[p] for p in predictions]
p, r, f1 = scorer.score(batch.gold(), predictions, verbose=True)

# save probability scores
if len(args.out) > 0:
    helper.ensure_dir(os.path.dirname(args.out))
    with open(args.out, 'wb') as outfile:
        pickle.dump(all_probs, outfile)
    print("Prediction scores saved to {}.".format(args.out))

print("Evaluation ended.")



Loading model from saved_models/00/best_model.pt
Finetune all embeddings.
Vocab size 421 loaded from file
Loading data from dataset/tacred/test.json with batch size 50...
1 batches created for dataset/tacred/test.json

Running with the following configs:
	data_dir : dataset/tacred
	vocab_dir : dataset/vocab
	emb_dim : 300
	ner_dim : 30
	pos_dim : 30
	hidden_dim : 200
	num_layers : 2
	dropout : 0.5
	word_dropout : 0.04
	topn : 10000000000.0
	lower : False
	attn : True
	attn_dim : 200
	pe_dim : 30
	lr : 1.0
	lr_decay : 0.9
	optim : sgd
	num_epoch : 30
	batch_size : 50
	max_grad_norm : 5.0
	log_step : 20
	log : logs.txt
	save_epoch : 5
	save_dir : ./saved_models
	id : 00
	info : 
	seed : 1234
	cuda : False
	cpu : False
	num_class : 42
	vocab_size : 421
	model_save_dir : ./saved_models/00


Per-relation statistics:
org:top_members/employees  P: 100.00%  R:   0.00%  F1:   0.00%  #: 1
per:age                    P: 100.00%  R:   0.00%  F1:   0.00%  #: 1
per:origin                 P: 100.00%  

In [14]:
constant.LABEL_TO_ID

{'no_relation': 0,
 'per:title': 1,
 'org:top_members/employees': 2,
 'per:employee_of': 3,
 'org:alternate_names': 4,
 'org:country_of_headquarters': 5,
 'per:countries_of_residence': 6,
 'org:city_of_headquarters': 7,
 'per:cities_of_residence': 8,
 'per:age': 9,
 'per:stateorprovinces_of_residence': 10,
 'per:origin': 11,
 'org:subsidiaries': 12,
 'org:parents': 13,
 'per:spouse': 14,
 'org:stateorprovince_of_headquarters': 15,
 'per:children': 16,
 'per:other_family': 17,
 'per:alternate_names': 18,
 'org:members': 19,
 'per:siblings': 20,
 'per:schools_attended': 21,
 'per:parents': 22,
 'per:date_of_death': 23,
 'org:member_of': 24,
 'org:founded_by': 25,
 'org:website': 26,
 'per:cause_of_death': 27,
 'org:political/religious_affiliation': 28,
 'org:founded': 29,
 'per:city_of_death': 30,
 'org:shareholders': 31,
 'org:number_of_employees/members': 32,
 'per:date_of_birth': 33,
 'per:city_of_birth': 34,
 'per:charges': 35,
 'per:stateorprovince_of_death': 36,
 'per:religion': 37